In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import os 
import numpy as np
import pandas as pd

In [2]:
path=r'F:\DiscovAir_data_explorer-master\DiscovAir_data_explorer-master\LNG3_P583\www\map1\tiles'

In [3]:
def compose_image(path):
    fils=os.listdir(path)
    #ylen=len(files)
    pathin=path+'/'+fils[2]
    ylen=len(os.listdir(pathin))
    pth=pathin+'/'+os.listdir(pathin)[0]
    files=os.listdir(pth)
    xlen=len(files)
    for p in files:
        IM=plt.imread(pth+'/'+p)
    pixelsize=IM.shape[0]
    dimensions=4
    image=np.zeros([xlen*pixelsize,ylen*pixelsize,dimensions])
    ee=0
    for e in os.listdir(pathin):
        pth=pathin+'/'+e
        ss=0
        for s in os.listdir(pth):
            IM=plt.imread(pth+'/'+s)
            image[(ss*pixelsize):(ss*pixelsize)+pixelsize,(ee*pixelsize):(ee*pixelsize)+pixelsize,:]=IM
            ss=ss+1
        ee=ee+1
    return image

In [4]:
image=compose_image(path)

In [5]:
for sectnum in range(1,8):
    coords=pd.read_csv('F:/Visium/LNG3_P583/Section'+str(sectnum)+'/583_LNG3_'+str(sectnum)+'_coords.csv')
    stereoscope=pd.read_csv('F:/Visium/LNG3_P583/Section'+str(sectnum)+'/583_LNG3_'+str(sectnum)+'_stereoscope.csv',index_col=0).transpose()
    coords['sample']=str(sectnum)
    adata=sc.AnnData(stereoscope)
    adata.obs=coords
    #adata.obs['name']=adata.obs['name'].astype(str)
    #adata.obs["name"]=adata.obs["name"].astype('category')
    adata.obsm["spatial"]=np.array([adata.obs.pixel_x,adata.obs.pixel_y]).transpose().astype('float64')
    #    sq.gr.spatial_neighbors(adata, spatial_key='spatial',radius=20)
    try: 
        adatall=sc.concat([adatall,adata],join='outer')
    except:
        adatall=adata

C:\Users\sergio.salas\AppData\Local\pymol\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  """
C:\Users\sergio.salas\AppData\Local\pymol\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
C:\Users\sergio.salas\AppData\Local\pymol\lib\site-packages\anndata\_core\anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
adatall.obs.index=adatall.obs.index.astype(str)
allstereo=pd.DataFrame(adatall.X)
allstereo.columns=adata.var.index
adata=adatall
smp=adatall[adatall.obs['sample']==adatall.obs['sample'].unique()[0]]
ss=smp.to_df()
col=pd.read_csv('F:/ISS/annotations_dicts/colors_celltype_annotated_general.csv',sep=';')
coldict=dict(zip(col['celltype_annotated_general'].astype(str),col['colors']))
coldict['SMG intermediate']='blue'


In [7]:
ss['VE']=np.sum(ss.loc[:,['VE-alveolar-capillary-Car4-aerocyte', 'VE-capillary',
       'VE-pulmonary-arterial', 'VE-pulmonary-venous', 'VE-systemic-arterial',
       'VE-systemic-venous']],axis=1)

In [8]:
ss.columns

Index(['AT1', 'AT2', 'B-mature', 'B-naive', 'B-plasma-IgA', 'B-plasma-IgG',
       'Basal', 'CD4', 'CD4-TRM/EMRA', 'CD4-naive/CM', 'CD8', 'CD8-CTL',
       'CD8-GZMK+', 'CD8-TRM', 'Chondrocyte', 'Ciliated', 'DC-1', 'DC-2',
       'DC-activated', 'DC-plasmacytoid', 'Endothelia-Lymphatic',
       'Fibro-adventitial', 'Fibro-alveolar', 'Fibro-myofibroblast',
       'Fibro-peribronchial', 'Fibro-perichondrial', 'ILC', 'MAIT',
       'Macro-AW-CX3CR1', 'Macro-CCL', 'Macro-CHIT1', 'Macro-alv-MT',
       'Macro-alveolar', 'Macro-interstitial', 'Macrophage-intermediate',
       'Macrophage-intravascular', 'Mast-cell', 'Monocyte-CD14',
       'Monocyte-CD16', 'Muscle-pericyte-pulmonary',
       'Muscle-perivascular-immune-recruiting', 'Muscle-smooth-airway-ASM',
       'Muscle-smooth-pulmonary', 'Muscle-smooth-systemic-arterial', 'NK',
       'NKT', 'NK-CD11d', 'NK-CD56bright', 'SMG-Duct', 'SMG-Mucous',
       'SMG-Serous', 'Secretory-Club', 'Secretory-Goblet', 'Suprabasal',
       'T-reg', 'VE

In [9]:
coldict['VE-capillary']='yellow'
coldict['VE-systemic-arterial']='yellow'
coldict['VE']='yellow'
coldict['Chondrocyte']='green'
coldict['Muscle-smooth-airway-ASM']='magenta'
coldict['SMG-Duct']='blue'
coldict['Secretory-Club']='red'
coldict['Secretory-Goblet']='red'
coldict['SMG-Serous']='blue'



In [ ]:
plt.figure(figsize=(int(image.shape[0]/300),int(image.shape[1]/300)),dpi=500)
plt.imshow(image)
cts=['SMG-Serous','Basal','Suprabasal','Chondrocyte','VE','Muscle-smooth-airway-ASM','Secretory-Club','Secretory-Goblet']
#plt.figure(figsize=(int(image.shape[0]/300),int(image.shape[1]/300)),dpi=500)
for ct in cts:
    plt.figure()
    plt.scatter(smp.obs['pixel_x']*4,smp.obs['pixel_y']*4,alpha=ss[ct],c=coldict[ct],s=15,linewidth=0)
#    plt.title(ct)
    plt.xlim(0,image.shape[0])
    plt.ylim(0,image.shape[1])
    plt.title(ct+'_'+str(np.round(np.max(ss[ct]),1)))
    plt.savefig(r'F:\ISS\plots_for_publication\visium/plot_sp_lng5_'+str(cts)+'.pdf')

In [ ]:
plt.figure(figsize=(int(image.shape[0]/300),int(image.shape[1]/300)),dpi=500)
plt.imshow(image)
cts=['SMG-Serous','Basal','Suprabasal','Chondrocyte','VE','Muscle-smooth-airway-ASM','Secretory-Club','Secretory-Goblet']
#plt.figure(figsize=(int(image.shape[0]/300),int(image.shape[1]/300)),dpi=500)
for ct in cts:
    plt.scatter(smp.obs['pixel_x']*4,smp.obs['pixel_y']*4,alpha=ss[ct],c=coldict[ct],s=15,linewidth=0)
    plt.title(ct)
    plt.xlim(0,image.shape[0])
    plt.ylim(0,image.shape[1])
plt.savefig(r'F:\ISS\plots_for_publication\visium/plot_sp.pdf')

In [ ]:
plt.figure(figsize=(int(image.shape[0]/300),int(image.shape[1]/300)),dpi=500)
plt.imshow(image)
cts=['Basal','Suprabasal','Club','VE','Muscle-smooth-airway-ASM','SMG-Duct']
for ct in cts:
    plt.scatter(smp.obs['pixel_x']*4,smp.obs['pixel_y']*4,alpha=ss[ct],c=coldict[ct],s=15,linewidth=0)
    plt.title(ct)
    plt.xlim(0,image.shape[0])
    plt.ylim(0,image.shape[1])
plt.savefig(r'F:\ISS\plots_for_publication\visium/plot_sp.pdf')

In [ ]:
plt.figure(figsize=(int(image.shape[0]/300),int(image.shape[1]/300)),dpi=500)
plt.imshow(image)
cts=['Basal','Ciliated','Suprabasal','VE-pulmonary-arterial']
for ct in ss.columns:#cts:
    try:
        plt.figure()
        plt.scatter(smp.obs['pixel_x']*4,smp.obs['pixel_y']*4,alpha=ss[ct],c=coldict[ct],s=20,linewidth=0)
        plt.title(ct)
    except:
        plt.figure()
        plt.scatter(smp.obs['pixel_x']*4,smp.obs['pixel_y']*4,alpha=ss[ct],c='red',s=20,linewidth=0)
        plt.title(ct)